In [1]:
# Importation
import numpy as np
import pandas as pd
import plotly.express as px
from  rouge_score  import  rouge_scorer
import re
from difflib import SequenceMatcher

# **I - Data management**

In [11]:
# Chargement des données
df = pd.read_csv("../data/01_251021_Notations200.csv")

## Présentation des données 


Le jeu de données qu'on traite dans le notebook est issue du jeu de données open source  [La fiscalité et les dépenses publiques](https://www.data.gouv.fr/fr/datasets/r/bc085888-e6bd-445d-b3f4-632190c29e3f). Celui-ci est notament constitué des réponses textuelles des contribuables à des questions portant sur la fiscalité en France. Dans cette étude, on se penche principalement sur la question suivante  : *Que faudrait-il faire pour rendre la fiscalité plus juste et plus efficace ?*.

Pour extraire idées majeures des réponses textuelles, on utilise le modèle de langue ollama : *hugging-quants/Meta-Llama-3.1-8B-Instruct-AWQ-INT4*. Avec les hyperparamètres **top_p = 0.95**
et **temperature = 0**.

On donne à ce LLM le prompt suivant :


[prompt]
system = """
But: extraire les idées principales DISTINCTES d’un texte pour analyse.

Règles:
1. N’utiliser QUE le contenu entre <<< TEXT >>>.
2. Extraire la liste des idées DISTINCTES et PRINCIPALES.
   - Chaque idée = une phrase claire, autonome, reformulée si nécessaire.
3. Pour CHAQUE idée, annoter:
   - type: "statement" (constat) OU "proposition" (suggestion/recommandation/objectif).
   - syntax: "negative" si la phrase contient une négation explicite (ex.: "ne", "n'", "ne pas", "ne plus", "non"), sinon "positive".
   - semantic: "positive", "negative" ou "neutral" (valence sémantique).
4. Sortie STRICTEMENT en CSV avec entête EXACTE:
   CSV:description,type,syntax,semantic
   - Délimiteur: virgule.
   - Chaque description entre guillemets doubles.
   - Échapper tout guillemet interne par duplication (ex.: ""chat"").
   - NE RIEN AJOUTER d’autre (pas de texte avant/après, pas de code fences).
   - Pas de lignes vides.

Exemple:
CSV:description,type,syntax,semantic
"Les chats retombent sur leurs pattes",statement,positive,neutral
"Les chats n'ont pas neuf vies",statement,negative,negative
"Il faut mieux prendre soin des chats pour prolonger leur vie",proposition,positive,positive
"""
user = """<<< {input} >>>"""

Ainsi, on construit le jeu de données **df**. Ces variables sont les suivantes :

- **authorId** : Identifiant de l’auteur
- **contrib_index** : Numéro de la contribution
- **contribution** : Texte original de l'auteur
- **C** : Score calculé par la métrique *QualIT*
- **n_ideas** : Nombre d’idées extraites
- **len_contrib** : Longueur du texte original
- **Matthias, Yannis, Garance** : Note allant de 0 à 10 évaluant selon la personne la qualité de l'extraction
- **Hallucinations** : Présence d'hallucination dans l'extraction
- **Idées_non_ind** : Idées dénuée de sens dans l'extraction

On construit ensuite de nouvelles variables à partir de ces données brutes pour faciliter l'analyse :

In [12]:
# Calcul du score humain moyen
df["score_humain"] = df[["Garance", "Matthias", "Yannis"]].mean(axis=1)

# Copie des colonnes d'hallucinations/d'idées invalides en version catégorielle
mapping = {0:"Non", 1:"Oui"}
df["pres_hallu"] = df["Hallucinations"].map(mapping)
df["pres_idees_inv"] = df["Idées_non_ind"].map(mapping)

# Nombre de tokens par contribution et par extractions (comptage simple par espaces)
df["nb_tokens_contrib"] = df["contribution"].apply(lambda x: len(str(x).split()))
df["nb_tokens_extraction"] = df["ideas_text"].apply(lambda x: len(str(x).split()))

# **II - Validité de la notation et de la métrique**

In [ ]:
# Calcul des corrélations de Pearson entre chaque paire d'évaluateurs
corr_GM = df["Garance"].corr(df["Matthias"], method='pearson')
corr_GY = df["Garance"].corr(df["Yannis"], method='pearson')
corr_MY = df["Matthias"].corr(df["Yannis"], method='pearson')
# Corrélation de Pearson entre score humain et score QualIT
correlation = df["score_humain"].corr(df["C"], method='pearson')

print(f"Corrélation (de Pearson) entre Garance et Matthias : {corr_GM:.2f}")
print(f"Corrélation (de Pearson) entre Garance et Yannis : {corr_GY:.2f}")
print(f"Corrélation (de Pearson) entre Matthias et Yannis : {corr_MY:.2f}")

Corrélation (de Pearson) entre Garance et Matthias : 0.93
Corrélation (de Pearson) entre Garance et Yannis : 0.93
Corrélation (de Pearson) entre Matthias et Yannis : 0.94
Corrélation entre le score humain moyen et le score QualIT : 0.71


Les trois évaluateurs humains (Matthias, Yannis, Garance) semblent globalement en accord.

# **III - Visualisations**

### **A - Caractéristiques des contributions**

In [10]:
# Barplot du nombre de charactères par contribution sur les 200 premières contributions
fig = px.histogram(
    df, x="nb_tokens_contrib",
    color_discrete_sequence=["#2a2781"]
)
fig.update_layout(
    title={"text": "<b>Distribution du nombre de tokens par contribution</b>"}, 
    xaxis_title="Nombre de tokens", 
    yaxis_title="Nombre de contributions", 
    width=800, height=450
)
fig.update_traces(
    hovertemplate =
        "Nombre de tokens = %{x}<br>" \
        "Nombre de contributions = %{y}<extra></extra>"
)
fig.show()

Au vu de la distribution, il semble judicieux de regrouper les contributions en différentes catégories de longueur (arbitraire).

In [11]:
# Répartition des contributions en 5 catégories équilibrées selon le nombre de tokens (basé sur les quantiles)
df["contrib_tokens_bins"] = pd.qcut(df["nb_tokens_contrib"], q=5, labels=[
    "Très court", "Court", "Moyen", "Long", "Très long"
])

# Autre options : catégories basées sur des seuils fixes
df["contrib_tokens_bins_fixe"] = pd.cut(df["nb_tokens_contrib"], bins=[0, 50, 100, 200, np.inf], labels=[
    "Très court (0-50)", "Court (51-100)", "Moyen (101-200)", "Long (+200)"
])

### **B - Caractéristiques des extractions**

In [12]:
# Version avec les bins équilibrés
fig1 = px.histogram(
    df, x="contrib_tokens_bins", y="Hallucinations",
    color_discrete_sequence=["#2a2781"], histfunc="avg", category_orders={
        "contrib_tokens_bins": ["Très court", "Court", "Moyen", "Long", "Très long"]
    }
)
fig1.update_layout(
    title={"text": 
        "<b>Distribution du taux d'hallucinations par catégories <br>équilibrées de longueur</b>"
    }, 
    xaxis_title="Catégories de longueur (en nombre de tokens)", yaxis_title="Taux d'hallucinations",
    width=700, height=450
)
fig1.update_traces(
    hovertemplate =
        "Taux d'hallucinations = %{y}<extra></extra>"
)
# Version avec les bins fixes
fig2 = px.histogram(
    df, x="contrib_tokens_bins_fixe", y="Hallucinations",
    color_discrete_sequence=["#2a2781"], histfunc="avg", category_orders={
        "contrib_tokens_bins_fixe": ["Très court (0-50)", "Court (51-100)", "Moyen (101-200)", "Long (+200)"]
    }
)
fig2.update_layout(
    title={"text": 
        "<b>Distribution du taux d'hallucinations par catégories <br>arbitraire de longueur</b>"
    }, 
    xaxis_title="Catégories de longueur (en nombre de tokens)", yaxis_title="Taux d'hallucinations",
    width=700, height=450
)
fig2.update_traces(
    hovertemplate =
        "Taux d'hallucinations = %{y}<extra></extra>"
)

# Affichage des figures
fig1.show()
fig2.show()

In [13]:
# Version avec les bins équilibrés
fig1 = px.histogram(
    df, x="contrib_tokens_bins", y="Idées_non_ind",
    color_discrete_sequence=["#2a2781"], histfunc="avg", category_orders={
        "contrib_tokens_bins": ["Très court", "Court", "Moyen", "Long", "Très long"]
    }
)
fig1.update_layout(
    title={"text": 
        "<b>Distribution du taux d'idées invalides par catégories <br>équilibrées de longueur</b>"
    }, 
    xaxis_title="Catégories de longueur (en nombre de tokens)", yaxis_title="Taux d'idées invalides",
    width=700, height=450
)
fig1.update_traces(
    hovertemplate =
        "Taux d'idées invalides = %{y}<extra></extra>"
)
# Version avec les bins fixes
fig2 = px.histogram(
    df, x="contrib_tokens_bins_fixe", y="Idées_non_ind",
    color_discrete_sequence=["#2a2781"], histfunc="avg", category_orders={
        "contrib_tokens_bins_fixe": ["Très court [0;50]", "Court [51;100]", "Moyen [101;200]", "Long +200"]
    }
)
fig2.update_layout(
    title={"text": 
        "<b>Distribution du taux d'idées invalides par catégories <br>arbitraire de longueur</b>"
    }, 
    xaxis_title="Catégories de longueur (en nombre de tokens)", yaxis_title="Taux d'idées invalides",
    width=700, height=450
)
fig2.update_traces(
    hovertemplate =
        "Taux d'idées invalides = %{y}<extra></extra>"
)

# Affichage des figures
fig1.show()
fig2.show()